In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import PIL.Image as pil_image

In [2]:
model = keras.applications.ResNet101(include_top=False, input_shape=(45, 45, 3));
model.summary();

Metal device set to: Apple M1


2022-10-02 02:28:17.905053: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-02 02:28:17.905340: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 45, 45, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 51, 51, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 23, 23, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 23, 23, 64)   256         ['conv1_conv[0][0]']             
                                                                                          

In [3]:
for layer in model.layers:
    layer.trainable = False

model = keras.Model(inputs=model.inputs, outputs=model.outputs)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 45, 45, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 51, 51, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 23, 23, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 23, 23, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

In [4]:
flatten = keras.layers.Flatten()(model.layers[-1].output);
output = keras.layers.Dense(82, activation=keras.activations.softmax)(flatten)

model = keras.Model(inputs = model.inputs, outputs=output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 45, 45, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 51, 51, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 23, 23, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 23, 23, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

In [2]:
import os
classifierList = os.listdir('input')
classifierList.remove('.DS_Store')
print(classifierList)
print(len(classifierList))

['beta', 'pm', 'Delta', 'gamma', 'infty', 'rightarrow', 'div', 'gt', 'forward_slash', 'leq', 'mu', 'exists', 'in', 'times', 'sin', 'R', 'u', '9', '0', '{', '7', 'i', 'N', 'G', '+', ',', '6', 'z', '}', '1', '8', 'T', 'S', 'cos', 'A', '-', 'f', 'o', 'H', 'sigma', 'sqrt', 'pi', 'int', 'sum', 'lim', 'lambda', 'neq', 'log', 'ldots', 'forall', 'lt', 'theta', 'ascii_124', 'M', '!', 'alpha', 'j', 'C', ']', '(', 'd', 'v', 'prime', 'q', '=', '4', 'X', 'phi', '3', 'tan', 'e', ')', '[', 'b', 'k', 'l', 'geq', '2', 'y', '5', 'p', 'w']
82


In [3]:
print(classifierList[18])

0


In [7]:
def to_one_hot(index):
    results = np.zeros(82)
    results[index] = 1
    return results;

one_hot = []
for i in range (0, 82):
    one_hot.append(to_one_hot(i))

print(len(one_hot))
print(one_hot[0])

82
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
X = []
y = []
testX = []
testy = []
j = 0
for i in range (0, 82):
    ImList = os.listdir('input/' + classifierList[i])
    fileDir = 'input/' + classifierList[i] + '/'
    k=0
    for str in ImList:
        image = keras.preprocessing.image.load_img(fileDir + str, target_size=(45, 45))
        image = keras.preprocessing.image.img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = keras.applications.vgg19.preprocess_input(image)
        if k < 1000:
            X.append(image)
            y.append(one_hot[i])
        
        if k < 2000:
            testX.append(image)
            testy.append(one_hot[i])
            
        if k == 2000:
            break
        k += 1
        j = j + 1
        if j % 1000 == 0:
            print(f'{j}' + "image processed");
            print(len(X))

print(len(X))


1000image processed
1000
2000image processed
1000
3000image processed
2000
4000image processed
3000
5000image processed
3348
6000image processed
4217
7000image processed
4514
8000image processed
5514
9000image processed
6514
10000image processed
7514
11000image processed
7891
12000image processed
8514
13000image processed
8891
14000image processed
9514
15000image processed
9891
16000image processed
10514
17000image processed
11245
18000image processed
11891
19000image processed
12245
20000image processed
12891
21000image processed
13245
22000image processed
14245
23000image processed
14267
24000image processed
15245
25000image processed
15267
26000image processed
16245
27000image processed
16267
28000image processed
17245
29000image processed
17553
30000image processed
18267
31000image processed
18553
32000image processed
19267
33000image processed
19647
34000image processed
20267
35000image processed
20647
36000image processed
21267
37000image processed
21647
38000image processed
2264

In [9]:
X = np.vstack(X)
print('X Vstack complete')
y = np.vstack(y)
print('Y Vstack complete')


X Vstack complete
Y Vstack complete


In [10]:
pip install tensorflow-macos

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install tensorflow-metal

Note: you may need to restart the kernel to use updated packages.


In [12]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
model.fit(X[0:100000], y[0:100000], epochs=5)

Epoch 1/5


2022-10-02 02:28:51.240766: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-02 02:28:54.143966: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2112/2112 [==============================] - 127s 58ms/step - loss: 1.1552 - accuracy: 0.7606
Epoch 2/5
2112/2112 [==============================] - 124s 59ms/step - loss: 0.6006 - accuracy: 0.8779
Epoch 3/5
2112/2112 [==============================] - 125s 59ms/step - loss: 0.4704 - accuracy: 0.9089
Epoch 4/5
2112/2112 [==============================] - 132s 63ms/step - loss: 0.3785 - accuracy: 0.9245
Epoch 5/5
2112/2112 [==============================] - 138s 65ms/step - loss: 0.3272 - accuracy: 0.9368


In [13]:
model.evaluate(X[0:100000], y[0:100000])

2022-10-02 02:39:41.463556: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2112/2112 [==============================] - 132s 62ms/step - loss: 0.2492 - accuracy: 0.9516


[0.2491849660873413, 0.9516298174858093]

In [14]:
testX=np.vstack(testX)
print('vstack x compl')
testy=np.vstack(testy)
print('vstack y compl')
model.evaluate(testX[0:100000], testy[0:100000])

vstack x compl
vstack y compl
3125/3125 [==============================] - 210s 67ms/step - loss: 0.4092 - accuracy: 0.9320


[0.4091682434082031, 0.9320200681686401]

In [15]:
model.save('proto2-resnet101')

INFO:tensorflow:Assets written to: proto2-resnet101/assets


INFO:tensorflow:Assets written to: proto2-resnet101/assets
